In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
seasons = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MSeasons.csv')
results_regular = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv') #../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyCompactResults.csv
results_tourney = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
teams = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MTeams.csv')

In [ ]:
results = pd.concat([results_regular, results_tourney], axis=0)
results = results.sort_values(['Season', 'DayNum'])

In [ ]:
# Dict keeping track of current rating
# team_dict with initial ratings
team_dict = {}
for team in teams.TeamID:
    team_dict[team] = {'rating': 2000}

# dictionary storing the rating at the beginning of each season/ the end of the last season
ranking_season_start = {}

In [ ]:
for ind, row in tqdm(results.iterrows()):
    if row['Season'] not in ranking_season_start:
        ranking_season_start[row['Season']] = {}
    if row['WTeamID'] not in ranking_season_start[row['Season']]:
        ranking_season_start[row['Season']][row['WTeamID']] = team_dict[row['WTeamID']].copy()
    if row['LTeamID'] not in ranking_season_start[row['Season']]:
        ranking_season_start[row['Season']][row['LTeamID']] = team_dict[row['LTeamID']].copy()
    rateW = 1 / (1+10**((team_dict[row['LTeamID']]['rating']-team_dict[row['WTeamID']]['rating'])/2000)) # expected result for W-Team
    rateL = 1 / (1+10**((team_dict[row['WTeamID']]['rating']-team_dict[row['LTeamID']]['rating'])/2000)) # expected result for L-Team
    #modifier = row['WScore'] / (row['WScore'] + row['LScore'] ) # modifier to award more points for larger margin
    team_dict[row['WTeamID']]['rating'] += 140*(1-rateW) # new rating W
    team_dict[row['LTeamID']]['rating'] += 140*(0-rateL) # new rating L
    if team_dict[row['LTeamID']]['rating'] < 1:
        team_dict[row['LTeamID']]['rating'] = 1

In [ ]:
f_ratings = [team_dict[key]['rating'] for key in team_dict]
sns.kdeplot(f_ratings)
plt.show()

In [ ]:
submission = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')

pred = []
for ind, row in tqdm(submission.iterrows()):
    g_id = row['ID'].split('_')
    rating1 = 2000
    rating2 = 2000
    if int(g_id[1]) in ranking_season_start[int(g_id[0])]:
        rating1 = ranking_season_start[int(g_id[0])][int(g_id[1])]['rating']
    if int(g_id[2]) in ranking_season_start[int(g_id[0])]:
        rating2 = ranking_season_start[int(g_id[0])][int(g_id[2])]['rating']
    rate = 1 / (1+4**((rating2-rating1)/2000))
    pred.append(rate)

In [ ]:
sns.kdeplot(pred)
plt.show()

In [ ]:
submission['Pred'] = pred
submission.to_csv('submission.csv', index=False)
submission.head()